In [7]:
import numpy as np
import os

path_txt_orig = '/lab/tmpig21/u/arios/8dset_shell/shell_experiments/8dset/Shell_HalfClasses_8dset/'

path_txt_out = '/lab/tmpig21/u/arios/8dset_shell/shell_experiments/8dset/Shell_agents_5classeach_8dset/'

root_svhn = '/lab/arios/ProjIntel/incDFM/data/svhn/'


# read by line numbers
def get_lines(fp, line_numbers):
    return (x for i, x in enumerate(fp) if i in line_numbers)



# def read_SVHN_labels(root, subset, train=True):

#     root = os.path.expanduser(root)
    
#     if train==True:
#         filename = "train_32x32.mat"
#     else:
#         filename = "test_32x32.mat"

        
#     import scipy.io as sio
#     # reading(loading) mat file as array
#     loaded_mat = sio.loadmat(os.path.join(root, filename))
#     # loading from the .mat file gives an np array of type np.uint8
#     # converting to np.int64, so that we have a LongTensor after
#     # the conversion from the numpy array
#     # the squeeze is needed to obtain a 1D tensor
#     labels = loaded_mat['y'].astype(np.int64).squeeze()

#     # the svhn dataset assigns the class label "10" to the digit 0
#     # this makes it inconsistent with several loss functions
#     # which expect the class labels to be in the range [0, C-1]
#     np.place(labels, labels == 10, 0)
    
#     labels = labels[subset]
      
#     return labels




In [8]:

split= 'validation'
files_per_dataset = os.listdir(path_txt_orig+split+'/')


num_per_agent=5

for file in files_per_dataset:
    file = '%s/%s/%s'%(path_txt_orig,split,file)
    task_agent = int(file.split('_')[-1].split('.')[0])
    if 'txt' in file:
        
        # continue
            
        if task_agent==1:
            classes = np.genfromtxt(file, delimiter='$').T[-1].astype(int)
        else:
            classes = np.genfromtxt(file, delimiter=' ').T[-1].astype(int)
            
        all_classes = np.unique(classes)
        
        class_splits = np.array_split(all_classes.tolist(), int(len(all_classes.tolist())/num_per_agent))
        
        
        # ---- Inner loop for each agent 
        for agent, classes_agent in enumerate(class_splits):
            inds_agent = np.where(np.in1d(classes, classes_agent))[0]
            
            filename_out = '%s/%s/nc_%s_task_%d_%d.txt'%(path_txt_out, split, split, task_agent, agent)
            
            with open(file, 'r') as fp:
                
                # read line 4 and 7
                lines = get_lines(fp, inds_agent.tolist())
                
                lines = [l for l in lines]
                
                with open(filename_out, 'a') as outfp:
                    outfp.writelines(lines)

    else:
        if split=='validation' or split=='test':
            continue

        filename_out = '%s/%s/nc_%s_task_%d_%d.npy'%(path_txt_out, split, split, task_agent, 0)
    
        inds = np.load(file)
        
        np.save(filename_out, inds)
        # break
